In [1]:
# set configurations
# from model.config import data_dir, weights_load_path

# base_dir = "."
# data_dir = "../../datasets/coco"  
# weights_load_path = "./model/weights/weight_stepLR_2.pth"
# activation_base_dir = "."


In [1]:
import json
from model.resnet18 import resnet18
from types import SimpleNamespace

In [2]:

EXPERIMENT_LOCAL_CONFIG = "./local_config.json"
with open(EXPERIMENT_LOCAL_CONFIG) as f:
    json_dict = json.load(f)
param = SimpleNamespace(**json_dict)

In [3]:
model = resnet18(param.pretrain, param.class_ids, param.finetune,param.weights_load_path, param.end_epoch, 
                 param.start_epoch, param.checkpoints, param.save_weights_loc)

Using cache found in /Users/pawnesh/.cache/torch/hub/pytorch_vision_v0.6.0


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  